In [1]:
#This validation code serves the following purposes
#1. perform a sanity check. binary search method and Ben-tal's method should return the same gradient and objective value.
#2. check that when Gamma<3, outputs from binary search method are optimal, by checking KKT conditions (grad is a multiple of np.ones(n))
#3. show that when Gamma<3, Dankins' Theorem gradient match finite central diff gradient. 
#4. show that at termination of Frank Wolfe, we get the same weights by using bisection or Ben-tal's method.

In [2]:
from gurobipy import *
import numpy as np
import math
from sklearn.gaussian_process.kernels import RBF
from scipy.optimize import minimize
import time
import scipy

import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoLarsCV
import statistics
import time
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils.random import sample_without_replacement

t = time.time()
#parameters
priceMultiplier=1#multiplier of price in kernel
Gamma=0.05#100
P_low=5
P_high=10

sigma=1#bandwidth of kernel
dim=1#dim of feature X

def policy_to_evaluate(x,p):
    return 1.1*p

def expected_demand(x_arr,p_arr):
    #d_arr=(x_arr*7-p_arr+P_high)/(P_high-P_low+7)
    d_arr=(5*x_arr+3*(P_high-p_arr)-10)/10
    d_arr=np.minimum(d_arr,1)
    d_arr=np.maximum(d_arr,0)
    return d_arr

def realized_demand(x_arr,p_arr,rndseed):
    np.random.seed(rndseed+52028)
    return np.random.binomial(1,expected_demand(x_arr,p_arr))
    
#generate training dataset
rndseed=927#2352
train_size=30
np.random.seed(rndseed+883)
X_train=np.random.uniform(0,1,train_size)
#np.random.seed(rndseed+5325)
#P_train=np.random.uniform(P_low,P_high,train_size)
P_train=P_high-(P_high-P_low)*X_train#deterministic pricing
D_train=realized_demand(X_train,P_train,rndseed+917)
R_train=np.multiply(D_train,P_train)
expected_R_train=np.multiply(P_train,expected_demand(X_train,P_train))

#generate counterfactuals
P_test=np.zeros(train_size)
for i in range(0,train_size):
    P_test[i]=policy_to_evaluate(X_train[i],P_train[i])
D_test=realized_demand(X_train,P_test,rndseed+1847)
realized_val=D_test@P_test/train_size
expected_R_test=np.multiply(P_test,expected_demand(X_train,P_test))
expected_val_test=sum(expected_R_test)/train_size

#for computing kernel only: normalize prices, and multiply by given factor
P_train_norm=priceMultiplier*(P_train-P_low)/ (P_high-P_low)

Z_train=np.zeros((train_size,dim+1))
for i in range(0,train_size):
    Z_train[i]=np.append(X_train[i],P_train_norm[i])#historical policy
Y_train=np.zeros((train_size,dim+1))
for i in range(0,train_size):
    Y_train[i]=np.append(X_train[i],policy_to_evaluate(X_train[i],P_train_norm[i]))#new policy

#gram matrix
kernel = RBF(sigma)
Z_and_Y=np.zeros((train_size*2,dim+1))
Z_and_Y[0:train_size,:]=Z_train
Z_and_Y[train_size:2*train_size,:]=Y_train
G=kernel(Z_and_Y)
G=G+0.0001*np.identity(train_size*2)#make G positive semi-definite
C=np.linalg.cholesky(G*2)#C is lower triangular, CC^\top=G

#preparation
rowSumHalfG=np.transpose(np.sum(G[:,train_size:2*train_size],axis=1).reshape(-1,1))#sum each row of right half of G
D3=np.transpose(rowSumHalfG).dot(rowSumHalfG)/(train_size**2)#the third term in Dw
arr_m=np.zeros((train_size,train_size*2,train_size*2))
for i in range(0,train_size):
    temp2=G[:,i].reshape(-1,1)
    arr_m[i]=temp2.dot(np.transpose(temp2))

    
print('preparation done')

preparation done


In [3]:
#Nathan's method
def nathans_method(lamda): 
    G_aug=np.copy(G)
    for i in range(0,train_size):
        G_aug[i][i]=G_aug[i][i]+lamda**2#add variance term

    #evals and evectors of G
    evals,evecs=scipy.linalg.eigh(G_aug, eigvals_only=False)#this method assumes real symmetric matrix, and is fast
    A=np.diag(evals)
    c=np.sum(evecs[0:train_size,0:2*train_size],axis=0)

    #build optimization model
    m = Model()
    m.Params.LogToConsole = 0#suppress Gurobipy printing
    # Add variables
    v=m.addMVar ( train_size*2, ub=1.0,lb=-1.0,name="v" )
    # Set objective function
    #m.Params.OptimalityTol=0.01
    m.setObjective(v@A@v, GRB.MINIMIZE) 
    # Add constraints
    m.addConstr(v@c==1)
    m.addConstr(evecs[train_size:2*train_size,0:2*train_size]@v==-np.ones(train_size)/train_size)
    #m.addConstr(S[0:train_size,0:2*train_size]@v>=-np.zeros(train_size))#w_i>=0
    #m.params.method=0#0 is primal simplex, 1 is dual simplex, 2 is barrier
    m.update()
    m.optimize()

    #print key info for testing purpose
    v_star=np.zeros(2*train_size)
    for i in range(0,2*train_size):
        v_star[i]=v[i].x
    w=evecs[0:train_size,0:2*train_size]@v_star

    #opt obj
    #obj = m.getObjective()
    #print('nathan opt obj: ', obj.getValue())
    
    return w

w_nathan_lambda_1=nathans_method(1)

Academic license - for non-commercial use only - expires 2022-09-08
Using license file C:\Users\yunfan\gurobi.lic


In [4]:
#subroutine that computes phi(w) using binary search
#input: weights. output: phi(w) and gradient
def subroutine_binary_search(weights):

    #compute D(w)
    wSumHalfG=(G[:,0:train_size].dot(weights)).reshape(-1,1)
    D2=2*wSumHalfG.dot(rowSumHalfG)/train_size#the second term in Dw
    D1=wSumHalfG.dot(np.transpose(wSumHalfG))#the first term in Dw
    for i in range(0,train_size):
        D1=D1-arr_m[i]*weights[i]**2
    Dw=-2*D1+D2+np.transpose(D2)-2*D3#Dw may not be PSD

    #compute S
    M=np.linalg.solve(C,Dw)#solve matrix M for CM=Dw
    B=np.linalg.solve(C,np.transpose(M))#solve matrix B for CB=M^\top

    evals,evecs=scipy.linalg.eigh(B, eigvals_only=False)#symmetric QR
    Q=np.copy(evecs)
    delta=np.copy(evals)
    S=np.linalg.solve(np.transpose(C),Q)
    
    #compute bw, epsilon
    temp2=np.multiply(np.multiply(weights,weights),P_train)
    bw=-G[:,0:train_size]@temp2
    epsilon=np.transpose(S).dot(bw)
    
    if delta[0]*delta[-1]>0:
        print('hessian is not indefinite')
    #binary search algorithm
    lambda_L=max(0,-min(delta))#ensure delta+lambda_L>=0
    lambda_U=100
    tol=0.000000001      
    while lambda_U-lambda_L>=tol:
        lambda_M=(lambda_U+lambda_L)/2
        x_star=np.divide(-epsilon,delta+lambda_M)
        if np.linalg.norm(x_star)>Gamma*(2**0.5):
            lambda_L=lambda_M
        else:
            lambda_U=lambda_M 
    optimal_dual=lambda_U
    x_star=np.divide(-epsilon,delta+optimal_dual) #recover opti primal variable from opti dual variable
    
    #get key values
    obj = np.multiply(delta,x_star)@x_star/2+epsilon@x_star
    obj_val = -obj #return phi(w) 
    q=S@x_star#worst case r()
    
    #compute gradient
    grad=np.zeros(train_size)
    for i in range(0,train_size):
        temp1=np.transpose(G[:,i]).reshape(-1,1)
        temp2=np.transpose(wSumHalfG)-weights[i]*np.transpose(temp1)-rowSumHalfG/train_size
        Hessian=2*temp1.dot(temp2)
        grad[i]=q.dot(Hessian.dot(np.transpose(q)))
        grad[i]=grad[i]+2*weights[i]*P_train[i]*np.transpose(G[:,i]).dot(q)
    
    return obj_val,grad

In [5]:
#compute gradients using finite central diffs
#input weights and objective value at this set of weights
def finite_centra_diff(weights,obj_1):
    finite_central_grad=np.zeros(train_size)
    h=0.0001
    for i in range(0,train_size):
        #w_temp=np.copy(weights)
        #w_temp[i]=w_temp[i]-h
        #obj_1,_=subroutine_binary_search(w_temp)
    
        w_temp=np.copy(weights)
        w_temp[i]=w_temp[i]+h
        obj_2,_=subroutine_binary_search(w_temp)

        finite_central_grad[i]=(obj_2-obj_1)/(h)
    return finite_central_grad

In [6]:
#subroutine that computes phi(w) using Ben-tal's method
#input: weights. output: phi(w) and gradient
def subroutine(weights):

    #compute D(w)
    wSumHalfG=(G[:,0:train_size].dot(weights)).reshape(-1,1)
    D2=2*wSumHalfG.dot(rowSumHalfG)/train_size#the second term in Dw
    D1=wSumHalfG.dot(np.transpose(wSumHalfG))#the first term in Dw
    for i in range(0,train_size):
        D1=D1-arr_m[i]*weights[i]**2
    Dw=-2*D1+D2+np.transpose(D2)-2*D3#Dw may not be PSD

    #compute S
    M=np.linalg.solve(C,Dw)#solve matrix M for CM=Dw
    B=np.linalg.solve(C,np.transpose(M))#solve matrix B for CB=M^\top

    evals,evecs=scipy.linalg.eigh(B, eigvals_only=False)#symmetric QR
    Q=np.copy(evecs)
    delta=np.copy(evals)
    S=np.linalg.solve(np.transpose(C),Q)
    
    #compute bw, epsilon
    temp2=np.multiply(np.multiply(weights,weights),P_train)
    bw=-G[:,0:train_size]@temp2
    epsilon=np.transpose(S).dot(bw)
    
    #build optimization model
    m = Model()
    m.Params.LogToConsole = 0#suppress Gurobipy printing
    # Add variables
    y=m.addMVar ( train_size*2, ub=100.0,lb=-100.0,name="y" )
    x=m.addMVar ( train_size*2, ub=100.0,lb=-100.0,name="x" )    
    # Set objective function
    #m.Params.OptimalityTol=0.01
    m.setObjective(epsilon@x+delta@y, GRB.MINIMIZE) 
    # Add constraints
    m.addConstr(np.ones(train_size*2)@y<=Gamma**2)
    m.addConstrs(x[i]@x[i]*0.5<=y[i] for i in range(train_size*2))
    m.params.method=1#0 is primal simplex, 1 is dual simplex, 2 is barrier
    #m.params.NonConvex=2
    m.update()
    m.optimize()
    
    #get key values
    obj = m.getObjective()
    obj_val = -obj.getValue() #return phi(w) 
    
    x_star=np.zeros(2*train_size)
    for i in range(0,2*train_size):
        x_star[i]=x[i].x
    q=S@x_star#worst case r()
    
    #compute gradient
    grad=np.zeros(train_size)
    for i in range(0,train_size):
        temp1=np.transpose(G[:,i]).reshape(-1,1)
        temp2=np.transpose(wSumHalfG)-weights[i]*np.transpose(temp1)-rowSumHalfG/train_size
        Hessian=2*temp1.dot(temp2)
        grad[i]=q.dot(Hessian.dot(np.transpose(q)))
        grad[i]=grad[i]+2*weights[i]*P_train[i]*np.transpose(G[:,i]).dot(q)
    
    return obj_val,grad

In [7]:
#check that bisection method and Ben-tal's method give the same gradient and objective value
for k in range(0,50):
    w_random=np.random.uniform(0,1,train_size)
    w_random=w_random/sum(w_random)
    obj_test1,grad_test1=subroutine(w_random)
    obj_test2,grad_test2=subroutine_binary_search(w_random)
    if abs(obj_test1-obj_test2)/obj_test1>0.005 or max(abs(np.divide(grad_test1-grad_test2,grad_test1)))>0.005:
        print('iterate: ',k,' not match','abs(obj_test1-obj_test2)= ',abs(obj_test1-obj_test2))
    

In [8]:
#main loop. Frank-Wolfe algorithm. Now use binary search
w=w_nathan_lambda_1
obj_val_old,grad=subroutine_binary_search(w) 
print('initial objective value: ',obj_val_old) 
#if Gamma>3:
#    grad=finite_centra_diff(w,obj_val_old)

k=1 
L=Gamma*100#a guess on lipschitz constant of phi(w) 
maxIter=1000
MSE_arr=np.zeros(maxIter+1) 
MSE_arr[0]=obj_val_old 
stepSize_arr=np.zeros(maxIter+1) 
step_size=1
gt=1
eta=0.1#parameter in Armijo rule
tau=0.8#parameter in Armijo rule
counter_small_steps=0
while (k<=maxIter and counter_small_steps<100):

    #compute descent direction
    #if Gamma>3:#when Gamma>3, Danskin's Theorem no longer apply
    #    grad=finite_centra_diff(w,obj_val_old)
    ind=np.argmin(grad)
    s_t=np.zeros(train_size)
    s_t[ind]=1
    descent_dir=s_t-w

    #compute initial step size
    gt=-grad@descent_dir
    step_size=min(gt/(L*descent_dir@descent_dir),1)
    
    #compute obj val and grad for initial step size
    obj_val_new,grad=subroutine_binary_search(w+step_size*descent_dir)
       
    #use Armijo rule for step size
    counter_Armijo=0
    while obj_val_old-obj_val_new<step_size*gt*eta and counter_Armijo<20:
        step_size=step_size*tau
        counter_Armijo+=1
        obj_val_new,grad=subroutine_binary_search(w+step_size*descent_dir)
        if counter_Armijo==20:
            counter_small_steps+=1
            if counter_small_steps==100:
                print('terminated before maxIter')
    
    obj_val_old=obj_val_new
    #store key outputs
    MSE_arr[k]=obj_val_new
    stepSize_arr[k]=step_size

    #update weights
    w=w+step_size*descent_dir
    if (k % 100==0):
        print('iter: ',k,'obj val: ',obj_val_new,'gt: ',gt,'min grad: ',min(grad),'coord: ',ind)
    k=k+1

#truncate dummy values
total_iter=k
MSE_arr=MSE_arr[0:k]
stepSize_arr=stepSize_arr[0:k]
print('runtime: ',time.time()-t) 
w_ours_binary_search=np.copy(w)

initial objective value:  0.0118439509580203
iter:  100 obj val:  0.011001011146267906 gt:  0.0037467531226543435 min grad:  0.01831075640807082 coord:  19
iter:  200 obj val:  0.010861271233299967 gt:  0.0018410944857810661 min grad:  0.019916701924151067 coord:  21
iter:  300 obj val:  0.010820294640463971 gt:  0.0011105912033937387 min grad:  0.02056087231781029 coord:  8
iter:  400 obj val:  0.010804434132598541 gt:  0.0007148897421997355 min grad:  0.020929448573352338 coord:  22
iter:  500 obj val:  0.010797131233530833 gt:  0.0005184320980270198 min grad:  0.021121158165591923 coord:  22
iter:  600 obj val:  0.010793169546102748 gt:  0.0003873487300103239 min grad:  0.02124447370041585 coord:  19
iter:  700 obj val:  0.010790822167503988 gt:  0.0003070228512695554 min grad:  0.021322468416846166 coord:  16
iter:  800 obj val:  0.010789277988083932 gt:  0.0002565644299150731 min grad:  0.021372834875971147 coord:  6
iter:  900 obj val:  0.010788198791717525 gt:  0.000215837750403

In [9]:
w_ours_binary_search@R_train

2.146427185010341

In [10]:
#check that gradient using Danskin's theorem and gradients using finite central diff are the same
#this check is performed on bisection method
w_t=np.copy(w)
_,grad=subroutine_binary_search(w_t)

finite_central_grad=np.zeros(train_size)
h=0.001
for i in range(0,train_size):
    w_temp=np.copy(w_t)
    w_temp[i]=w_temp[i]-h
    obj_1,_=subroutine_binary_search(w_temp)
    
    w_temp=np.copy(w_t)
    w_temp[i]=w_temp[i]+h
    obj_2,_=subroutine_binary_search(w_temp)
    #approx grad
    finite_central_grad[i]=(obj_2-obj_1)/(2*h)
print('finite central diff grad: ')
print(finite_central_grad)
print('envelope theorem grad: ')
print(grad)
print('percentage diff: ',(finite_central_grad-grad)/grad)

finite_central_grad_2=np.zeros(train_size)
for i in range(0,train_size):
    w_temp=np.copy(w_t)
    w_temp[i]=w_temp[i]-h
    obj_1,_=subroutine(w_temp)
    
    w_temp=np.copy(w_t)
    w_temp[i]=w_temp[i]+h
    obj_2,_=subroutine(w_temp)
    #approx grad
    finite_central_grad_2[i]=(obj_2-obj_1)/(2*h)

finite central diff grad: 
[0.0225621  0.02260835 0.02145721 0.02203038 0.02145341 0.02144991
 0.02144651 0.02146779 0.02144993 0.02204662 0.02145708 0.02213756
 0.02146349 0.02147422 0.02146685 0.021457   0.02145996 0.02219739
 0.02144476 0.02146312 0.02250346 0.021457   0.02145752 0.02145938
 0.02145712 0.02146015 0.02146278 0.02161204 0.02145327 0.02144563]
envelope theorem grad: 
[0.0225617  0.02260793 0.02145718 0.02203009 0.02145329 0.0214498
 0.02144637 0.02146775 0.02144981 0.02204632 0.02145698 0.02213725
 0.02146343 0.02147413 0.0214667  0.02145695 0.02145988 0.02219706
 0.02144468 0.021463   0.02250307 0.02145688 0.02145739 0.02145934
 0.0214571  0.02145998 0.02146267 0.02161181 0.02145316 0.02144546]
percentage diff:  [1.77933246e-05 1.83414930e-05 1.07717195e-06 1.33045897e-05
 5.43766134e-06 4.86014499e-06 6.49283102e-06 2.01436438e-06
 5.70070780e-06 1.33961200e-05 4.35051545e-06 1.40173586e-05
 2.64307050e-06 3.95915444e-06 6.75014038e-06 2.50209278e-06
 3.72952826e-06 

In [11]:
(finite_central_grad_2-finite_central_grad)/finite_central_grad

array([-3.05789517e-04,  6.78174006e-04,  1.62879834e-02,  4.36857144e-04,
        5.38330047e-04, -2.68297734e-04,  1.05389183e-02, -2.41389844e-02,
       -3.09522304e-03,  5.49238601e-04,  3.01828661e-02,  4.01025480e-04,
       -2.02688143e-02,  3.32703847e-02, -2.58592960e-03,  1.30480969e-03,
        6.18401233e-03, -1.09093657e-01, -1.20470801e-03, -2.07493148e-03,
        3.19553174e-03,  2.13617155e-05,  1.89575492e-03,  4.53663521e-04,
       -5.35034598e-05,  7.93560925e-04,  1.40323421e-02,  7.28487939e-03,
        1.46905236e-03, -3.93894569e-03])

In [12]:
#main loop. Frank-Wolfe algorithm. Using Ben-tal's subroutine
w=w_nathan_lambda_1
obj_val_old,grad=subroutine(w) 
print('initial objective value: ',obj_val_old) 
#if Gamma>3:
#    grad=finite_centra_diff(w,obj_val_old)

k=1 
L=Gamma*100#a guess on lipschitz constant of phi(w) 
maxIter=1000
MSE_arr=np.zeros(maxIter+1) 
MSE_arr[0]=obj_val_old 
stepSize_arr=np.zeros(maxIter+1) 
step_size=1
gt=1
eta=0.1#parameter in Armijo rule
tau=0.8#parameter in Armijo rule
counter_small_steps=0
while (k<=maxIter and counter_small_steps<100):

    #compute descent direction
    #if Gamma>3:#when Gamma>3, Danskin's Theorem no longer apply
    #    grad=finite_centra_diff(w,obj_val_old)
    ind=np.argmin(grad)
    s_t=np.zeros(train_size)
    s_t[ind]=1
    descent_dir=s_t-w

    #compute initial step size
    gt=-grad@descent_dir
    step_size=min(gt/(L*descent_dir@descent_dir),1)
    
    #compute obj val and grad for initial step size
    obj_val_new,grad=subroutine(w+step_size*descent_dir)
       
    #use Armijo rule for step size
    counter_Armijo=0
    while obj_val_old-obj_val_new<step_size*gt*eta and counter_Armijo<20:
        step_size=step_size*tau
        counter_Armijo+=1
        obj_val_new,grad=subroutine(w+step_size*descent_dir)
        if counter_Armijo==20:
            counter_small_steps+=1
            if counter_small_steps==100:
                print('terminated before maxIter')
    
    obj_val_old=obj_val_new
    #store key outputs
    MSE_arr[k]=obj_val_new
    stepSize_arr[k]=step_size

    #update weights
    w=w+step_size*descent_dir
    if (k % 100==0):
        print('iter: ',k,'obj val: ',obj_val_new,'gt: ',gt,'min grad: ',min(grad),'coord: ',ind)
    k=k+1

#truncate dummy values
total_iter=k
MSE_arr=MSE_arr[0:k]
stepSize_arr=stepSize_arr[0:k]
print('runtime: ',time.time()-t) 
w_ours=np.copy(w)

initial objective value:  0.011843755126783592
iter:  100 obj val:  0.011002743709307022 gt:  0.00374217608457675 min grad:  0.01831418893309774 coord:  19
iter:  200 obj val:  0.010861908372729595 gt:  0.0018493578900338169 min grad:  0.019921793739592665 coord:  23
iter:  300 obj val:  0.010821524406870417 gt:  0.0011298135323369856 min grad:  0.02054097626883627 coord:  26
iter:  400 obj val:  0.010807899874916059 gt:  0.0008184345946146022 min grad:  0.020826057757436102 coord:  16
iter:  500 obj val:  0.010800689946345253 gt:  0.0006232676648849945 min grad:  0.021020441635815647 coord:  2
iter:  600 obj val:  0.010796367040926815 gt:  0.000496813903972243 min grad:  0.0211290672959421 coord:  26
iter:  700 obj val:  0.010793864883809389 gt:  0.00042565746824868925 min grad:  0.021208114183506998 coord:  16
iter:  800 obj val:  0.01079251722775131 gt:  0.0003748486794344155 min grad:  0.021254785455782826 coord:  18
terminated before maxIter
runtime:  682.4900619983673


In [13]:
#check that gradient using Danskin's theorem and gradients using finite central diff are the same
#this check is performed on Ben-tal's method
w_t=np.copy(w_ours_binary_search)
obj_1,grad=subroutine(w_t)
approx_grad_arr=np.zeros(train_size)
h=0.0001
for i in range(0,train_size):
    w_temp=np.copy(w_t)
    w_temp[i]=w_temp[i]-h
    obj_1,_=subroutine(w_temp)
    
    w_temp=np.copy(w_t)
    w_temp[i]=w_temp[i]+h
    obj_2,_=subroutine(w_temp)
    #approx grad
    approx_grad_arr[i]=(obj_2-obj_1)/(2*h)
print('approx grad: ')
print(approx_grad_arr)
print('envelope theorem grad: ')
print(grad)
print('percentage diff: ',(approx_grad_arr-grad)/grad)

approx grad: 
[ 0.02263274  0.02299595  0.01948351  0.02416172  0.01987304  0.02161251
  0.02270028  0.02160432  0.02135592 -0.03876797  0.01393507  0.02021285
  0.02154924  0.02105198  0.02146526  0.02157159  0.02161349  0.02237629
  0.02167469  0.02144934  0.01994088  0.01961288  0.02085066  0.0191808
  0.02189582  0.01866441  0.12778892  0.02504859  0.02155588  0.02188996]
envelope theorem grad: 
[0.02255249 0.02259855 0.02145725 0.02202252 0.02144919 0.02144604
 0.02145287 0.02146609 0.02145589 0.02203871 0.02146225 0.02212939
 0.02146742 0.02147095 0.02146191 0.02146078 0.02146474 0.02218903
 0.0214495  0.021469   0.02249407 0.02146292 0.02146357 0.02145835
 0.02145823 0.02145472 0.02146822 0.02160528 0.02145908 0.02144016]
percentage diff:  [ 3.55823903e-03  1.75853365e-02 -9.19847360e-02  9.71368178e-02
 -7.34829355e-02  7.76231080e-03  5.81464604e-02  6.43959403e-03
 -4.65970288e-03 -2.75908503e+00 -3.50717323e-01 -8.66059117e-02
  3.81134110e-03 -1.95133384e-02  1.55982707e-04

In [14]:
w_ours

array([0.02899978, 0.02911225, 0.02879945, 0.02794006, 0.02646201,
       0.0265204 , 0.04721514, 0.02747709, 0.04466272, 0.02796795,
       0.04050039, 0.02812822, 0.03595433, 0.02668809, 0.0263085 ,
       0.03540741, 0.03885907, 0.02823775, 0.0386991 , 0.04404613,
       0.0288635 , 0.04436217, 0.04514516, 0.0278781 , 0.03011012,
       0.02641896, 0.04178464, 0.02728416, 0.04371561, 0.02645177])

In [15]:
#4. show that at termination of Frank Wolfe, we get the same weights by using bisection or Ben-tal's method.
w_ours_binary_search/w_ours

array([0.98091319, 0.98091319, 1.00834158, 0.98091319, 1.00885852,
       1.00935301, 1.0030188 , 1.00739759, 1.00309254, 0.98091319,
       1.00442707, 0.98091319, 1.00457294, 1.01073235, 1.01118399,
       1.00639287, 1.00491707, 0.98091319, 1.00485703, 1.005326  ,
       0.98091319, 1.00356584, 1.00416227, 1.00859363, 1.0059118 ,
       1.00509725, 1.00478736, 0.98091319, 1.00396055, 1.00307445])